# Heart Disease Artificial Intelligence Datathon 2021

**Baseline Code**

## 필독 TODO

* 해당 baseline 모델 및 오차 함수들은 Multiclass 및 One-Hot 라벨을 가정하고 만들어졌습니다.

* 따라서 그대로 사용하시기보단 loss function 튜닝이 필요합니다. (PR로 만들어주세요.)

* 그리고 dataset root path 설정해야 합니다.

# Runtime Preparation

## View Runtime Information

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0 or gpu_info.find('not found') >= 0:
    device = 'cpu'; print('Not connected to a GPU')
else: device = 'cuda'; print(gpu_info)

Thu Dec  2 03:37:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 470.74       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:01:00.0 Off |                  Off |
| 30%   36C    P0    28W / 230W |     10MiB / 24256MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:02:00.0 Off |                  Off |
| 30%   

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print(f'Your runtime has {ram_gb:.1f} gigabytes of available RAM\n'
      f'{"Not" if ram_gb < 20 else "You are"} using a high-RAM runtime!')

Your runtime has 67.2 gigabytes of available RAM
You are using a high-RAM runtime!


In [3]:
import os
import sys
import platform
import torch
print(f"OS version: \t\t{platform.platform()}\n"
      f"Python version:\t\t{sys.version.replace(chr(10), str())}\n"
      f"Torch version:\t\t{torch.__version__}\n"
      f"Torch device:\t\t{device}")

OS version: 		Linux-5.11.0-40-generic-x86_64-with-debian-buster-sid
Python version:		3.7.10 (default, Feb 26 2021, 18:47:35) [GCC 7.3.0]
Torch version:		1.9.0
Torch device:		cuda


## Prepare device and library

In [4]:
device = torch.device(device)


In [5]:
# After all installation, import all libraries used.

!pip install torchinfo
!pip install pyclean
!pyclean .

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import ConcatDataset, RandomSampler, DataLoader
from torchvision import transforms
import torchinfo


Cleaning directory .
Total 0 files, 0 directories removed.


# Dataset Preparation

## Make Dataset Class

In [6]:
from utils.dataset import ImageList

## Instantiate Dataset

In [7]:
root: str = "../echocardiography/"

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize((256, 256))])

train_a2c = os.path.join(root, 'train', 'A2C')
train_a2c = ImageList.from_path(train_a2c, transform=transform, target_transform=transform)

train_a4c = os.path.join(root, 'train', 'A4C')
train_a4c = ImageList.from_path(train_a4c, transform=transform, target_transform=transform)

val_a2c = os.path.join(root, 'validation', 'A2C')
val_a2c = ImageList.from_path(val_a2c, transform=transform, target_transform=transform)

val_a4c = os.path.join(root, 'validation', 'A4C')
val_a4c = ImageList.from_path(val_a4c, transform=transform, target_transform=transform)

train_datasets = ConcatDataset([train_a2c, train_a4c])
val_datasets = ConcatDataset([val_a2c, val_a4c])


# Network Preparation

## Segmentation Network

* **DeepLabV3 + Resnet101**: Baseline Model

* **U-Net**

* **Inception U-Net**

* **RefineNet**


In [8]:
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models.segmentation import deeplabv3_resnet101

from models.unet import UNet, InceptionUNet
from models.refinenet import refinenet50, refinenet101, refinenet152, rf_lw50, rf_lw101, rf_lw152

In [9]:
# # Baseline: DeeplabV3 + ResNet101

# # Pretrained Model
net = deeplabv3_resnet101(pretrained=True, progress=False)
# for grayscale image training - only 1 channel
net.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
net.classifier = DeepLabHead(2048, 1)
# net.aux_classifier = nn.Sequential()
net.aux_classifier = FCNHead(1024, 1)

# # Non-pretrained Model
# net = deeplabv3_resnet101(pretrained=False, num_classes=6)

trainable_backbone_layers = ['layer4']
for n, p in net.named_parameters():
    if n.startswith('backbone') and n.split('.')[1] not in trainable_backbone_layers:
        p.requires_grad = False

net.to(device)
if torch.cuda.device_count() > 1:
    net = torch.nn.DataParallel(net)
    net.to(device)

torchinfo.summary(net, (1, 1, 256, 256))

/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Layer (type:depth-idx)                                  Output Shape              Param #
DataParallel                                            --                        --
├─DeepLabV3: 1-1                                        [1, 1, 256, 256]          --
├─DeepLabV3: 1                                          --                        --
│    └─IntermediateLayerGetter: 2-1                     [1, 2048, 32, 32]         --
│    └─IntermediateLayerGetter: 2                       --                        --
│    │    └─Conv2d: 3-1                                 [1, 64, 128, 128]         (3,136)
│    │    └─BatchNorm2d: 3-2                            [1, 64, 128, 128]         (128)
│    │    └─ReLU: 3-3                                   [1, 64, 128, 128]         --
│    │    └─MaxPool2d: 3-4                              [1, 64, 64, 64]           --
│    │    └─Sequential: 3-5                             [1, 256, 64, 64]          (215,808)
│    │    └─Sequential: 3-6                  

## Loss Network

* **Binary Cross Entropy**

* **Dice Coefficient**

* **Intersection over Union Score**

- More Multi-Label Segmentation Losses: https://jeune-research.tistory.com/entry/Loss-Functions-for-Image-Segmentation-Region-Based-Losses

- See also: https://smp.readthedocs.io/en/latest/losses.html

In [10]:
from models.loss import BCEDiceIoUWithLogitsLoss2d, BCEDiceIoULoss2d

# Training

## Set Hyper Parameters

In [23]:
from utils.lr_scheduler import CosineAnnealingWarmUpRestarts

# Lazy-eval iterable dataset: do not set sampler or shuffle options
num_epoch = 10

batch_size = 35
num_workers = 1

loss_function = BCEDiceIoUWithLogitsLoss2d()
optimizer_class = torch.optim.Adam
optimizer_config = {'lr': 1e-6}
scheduler_class = CosineAnnealingWarmUpRestarts
scheduler_config = {'T_0': 10, 'T_mult': 2, 'eta_max': 1e-3, 'T_up': 3, 'gamma': 0.5}

## Train and Evaluate

In [38]:
train_loader = DataLoader(train_datasets, batch_size, num_workers=num_workers, drop_last=False)
val_loader = DataLoader(val_datasets, batch_size, num_workers=num_workers, drop_last=False)

optimizer = optimizer_class(net.parameters(), **optimizer_config)
lr_scheduler = scheduler_class(optimizer, **scheduler_config)


def load_state_dict(d):
    net.load_state_dict(d['model'])
    optimizer.load_state_dict(d['optimizer'])
    lr_scheduler.load_state_dict(d['lr_scheduler'])


def state_dict():
    from collections import OrderedDict
    d = OrderedDict()
    d['model'] = net.state_dict()
    d['optimizer'] = optimizer.state_dict()
    d['lr_scheduler'] = lr_scheduler.state_dict()
    return d


In [24]:
import uuid
from utils.training import train_one_epoch

try:
    print(f"Re-using session: {session_name}")
except NameError:
    session_name = str(uuid.uuid4())
    print(f"Generating session: {session_name}")

checkpoint_dir = f'checkpoint/{session_name}'
os.makedirs(checkpoint_dir, exist_ok=True)

for ep in range(num_epoch):
    train_one_epoch(net, loss_function, optimizer, lr_scheduler, train_loader, val_loader, device, ep, warmup_start=False)
    # Take care of computational resource.
    if ep == num_epoch - 1:
        torch.save(state_dict(), os.path.join(checkpoint_dir, '{}.pt').format(ep))

Re-using session: 59521cca-6dd4-4cb1-a2e8-a567d1d8dd7c


Iteration 0 train loss: 0.7780


Iteration 0 eval loss: 0.6034


Iteration 1 train loss: 0.7780


Iteration 1 eval loss: 0.6034


...

Iteration 7 eval loss: 0.6035


Iteration 8 train loss: 0.7779


Iteration 8 eval loss: 0.6030


Iteration 9 train loss: 0.7779


Iteration 9 eval loss: 0.6031


## Test

In [40]:
from utils.evaluation import all_together, draw_confusion_matrix

In [41]:
# TBD

label_names = [
    "Left Ventricle",
    "Background"
]

_, _, _, _, cm = all_together(net, val_loader, device=device, verbose=True)
draw_confusion_matrix(
    cm[:5, :5], label_names, label_names,
    figsize=(10, 8), title="Left Ventricle Division"
)

AttributeError: 'collections.OrderedDict' object has no attribute 'softmax'